## Get the Workspace

In [ ]:
from azureml.core.workspace import Workspace

workspace = Workspace.from_config()

## Set the Compute Target

In [ ]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

cluster_name = '<YOUR-CLUSTER-NAME>'
compute_target = ComputeTarget(workspace=workspace, name=cluster_name)

## Create an experiment

In [ ]:
from azureml.core import Experiment

experiment_name = 'churn-experiment'
experiment = Experiment(workspace=workspace, name=experiment_name)

## Create Environment

In [ ]:
from azureml.core import ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies

xgboost_env = Environment(workspace=workspace, name="XGBoost-Env")
conda_dep = CondaDependencies()

conda_dep.add_pip_package("azureml-dataprep[pandas]")
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("xgboost==0.90")
conda_dep.add_pip_package("scikit-learn==0.22.2.post1")
conda_dep.add_pip_package("numpy==1.18.5")
conda_dep.add_pip_package("seaborn")

# Adds dependencies to PythonSection of myenv
xgboost_env.python.conda_dependencies=conda_dep

In [ ]:
xgboost_env.register(workspace)

In [ ]:
project_folder = '../jobs/src'

# Non optimized Run
src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      arguments=['--max_depth', 30, 
                                 '--learning_rate', 0.5, 
                                 '--reg_alpha', 0.5, 
                                 '--reg_lambda', 0.2, 
                                 '--min_child_weight', 15.00],
                      compute_target=compute_target,
                      environment=xgboost_env)

# Optimized Run
# src = ScriptRunConfig(source_directory=project_folder,
#                       script='train.py',
#                       arguments=['--max_depth', 64, 
#                                  '--learning_rate', 0.2260, 
#                                  '--reg_alpha', 0.01, 
#                                  '--reg_lambda', 0.348, 
#                                  '--min_child_weight', 19.22],
#                       compute_target=compute_target,
#                       environment=xgboost_env)

## Submit the run

In [ ]:
run = experiment.submit(src)

## Monitor

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

## Register the Model

In [ ]:
run.download_file(name='outputs/model.pickle', output_file_path='outputs')

In [ ]:
model = run.register_model(model_name='churn-model', 
                           model_path='outputs/model.pickle',
                           description='Churn Model',
                           tags={'Model': 'XGBoost', 'type': 'Classification'},
                          )